In [6]:
# 引用需要的library
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from matplotlib import pyplot as plt
import helper
import joblib
import jieba

In [2]:
# 从文本文件中读数据
# 读取到的内容
# X=评论（string）， y=情感分类（0/1）
X, y = helper.Read_comments_from_file()
joblib.dump(value={"X": X, "y": y}, filename="data.pkl")
print("数据集长度：", len(X))
print("标签长度：", len(y))

数据集长度： 6000
标签长度： 6000


In [4]:
# 读取存储的语料
corpus = joblib.load("data.pkl")
X, y = corpus["X"], corpus["y"]
X[0], y[0]

('标准间太差房间还不如3星的而且设施非常陈旧.建议酒店把老的标准间从新改善.', 0)

In [24]:
# 调用jieba分词
# 文本向量化：1. 类one-hot编码 2. count思想 3. tf-idf思想
# 返回 X=评论向量（list），y=情感分类（0/1）

In [11]:
# 调用jieba分词
import jieba
X_ = []
for x in X:
    X_.append(" ".join(jieba.lcut(x)))

# 停用词
# 专用词库
        
X_[0]

'标准间 太 差 房间 还 不如 3 星 的 而且 设施 非常 陈旧 . 建议 酒店 把 老 的 标准间 从 新 改善 .'

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

# CounterVectorizer 向量化
cv = CountVectorizer()
data_X = cv.fit_transform(X_)

print("CountVectorizer 词汇字典：", cv.vocabulary_)
print("CountVectorizer 词汇列表：", cv.get_feature_names_out())
print(data_X.shape)

CountVectorizer 词汇字典： {'标准间': 13173, '房间': 10600, '不如': 1690, '而且': 17406, '设施': 18627, '非常': 21031, '陈旧': 20763, '建议': 9486, '酒店': 20229, '改善': 11670, '服务态度': 12804, '极其': 13063, '前台': 4895, '接待': 11330, '好象': 8024, '没有': 14039, '受过': 5800, '培训': 7135, '基本': 7139, '礼貌': 16282, '竟然': 16568, '同时': 6196, '几个': 4487, '客人': 8328, '大堂': 7488, '副理': 4948, '辩解': 19448, '没完': 14004, '总经理': 10134, '电话': 15592, '投诉': 10893, '不敢': 1766, '要是': 18372, '什么': 2951, '亏心事': 2678, '跟本': 19216, '不用': 1816, '这么': 19634, '地理位置': 7013, '不错': 1926, '哪里': 6560, '比较': 13638, '方便': 11968, '但是': 3303, '服务': 12794, '象是': 18899, '豪生': 18913, '集团': 20916, '管理': 16751, '下午': 1488, '一觉': 1168, '一个': 828, '本来': 12924, '打扫': 10738, '一下': 816, '所以': 10607, '打开': 10732, '可是': 5994, '晚上': 12438, '发现': 5729, '关掉': 4167, '还是': 19597, '马路': 21416, '房间内': 10601, '简陋': 16716, '并且': 9256, '玻璃窗': 15361, '户外': 10528, '还有': 19598, '一层': 971, '幕墙': 9166, '玻璃': 15357, '不能': 1851, '导致': 8528, '自然通风': 17696, '采光': 20267, '不好': 1687, '

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer 向量化
tf_vectorizer = TfidfVectorizer()
tfidf_X = tf_vectorizer.fit_transform(X_)

print("TfidfVectorizer 词汇字典：", tf_vectorizer.vocabulary_)
print("TfidfVectorizer 词汇列表：", tf_vectorizer.get_feature_names_out())
print(tfidf_X.shape)

TfidfVectorizer 词汇字典： {'标准间': 13173, '房间': 10600, '不如': 1690, '而且': 17406, '设施': 18627, '非常': 21031, '陈旧': 20763, '建议': 9486, '酒店': 20229, '改善': 11670, '服务态度': 12804, '极其': 13063, '前台': 4895, '接待': 11330, '好象': 8024, '没有': 14039, '受过': 5800, '培训': 7135, '基本': 7139, '礼貌': 16282, '竟然': 16568, '同时': 6196, '几个': 4487, '客人': 8328, '大堂': 7488, '副理': 4948, '辩解': 19448, '没完': 14004, '总经理': 10134, '电话': 15592, '投诉': 10893, '不敢': 1766, '要是': 18372, '什么': 2951, '亏心事': 2678, '跟本': 19216, '不用': 1816, '这么': 19634, '地理位置': 7013, '不错': 1926, '哪里': 6560, '比较': 13638, '方便': 11968, '但是': 3303, '服务': 12794, '象是': 18899, '豪生': 18913, '集团': 20916, '管理': 16751, '下午': 1488, '一觉': 1168, '一个': 828, '本来': 12924, '打扫': 10738, '一下': 816, '所以': 10607, '打开': 10732, '可是': 5994, '晚上': 12438, '发现': 5729, '关掉': 4167, '还是': 19597, '马路': 21416, '房间内': 10601, '简陋': 16716, '并且': 9256, '玻璃窗': 15361, '户外': 10528, '还有': 19598, '一层': 971, '幕墙': 9166, '玻璃': 15357, '不能': 1851, '导致': 8528, '自然通风': 17696, '采光': 20267, '不好': 1687, '

In [22]:
# 切分数据集为X_train, y_train, X_test, y_test
X_train, X_test, y_train, y_test = train_test_split(tfidf_X, y, test_size=0.2, random_state=0, shuffle=True)

In [ ]:
# 数据预处理：PCA降维？

In [ ]:
# 调用各分类模型，训练
# KNeighborsClassifier
# DecisionTreeClassifier
# SVC
# RandomForestClassifier
# AdaBoostClassifier
# GradientBoostingClassifier
# XGBClassifier
# LGBMClassifier

In [23]:
from sklearn.ensemble import RandomForestClassifier

# 构建模型
rfc = RandomForestClassifier()

# 训练模型
rfc.fit(X_train, y_train)

# 预测结果
# y_pred = rfc.predict(X_test)

# 模型评估
print(rfc.score(X=X_test, y=y_test))

0.8966666666666666


In [ ]:
# 调用训练完毕的模型，做预测

In [ ]:
# 计算各模型的acc，训练时间，预测时间

In [ ]:
# 保存模型到models文件夹